In [87]:
from pydantic_ai import Agent
from dotenv import load_dotenv
from datetime import datetime
from typing import List, Optional
from enum import Enum
from pydantic import BaseModel, Field, field_validator
from typing_extensions import Literal
import os 
import asyncio
import nest_asyncio 
nest_asyncio.apply()

In [88]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GEMINI_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [95]:
model="openai:gpt-4o-mini"
# model= 'gemini-1.5-flash'

Models 

In [96]:


class RequestType(str, Enum):
    CONSULTATION = "consultation"
    FOLLOW_UP = "follow_up"
    NEW_PATIENT = "new_patient"

class AppointmentDetails(BaseModel):
    patient_name: str
    doctor_id: str
    preferred_date: str = Field(..., description="Date in MM/DD/YYYY format")
    preferred_time: str = Field(..., description="Time in HH:MM AM/PM format")
    request_type: RequestType

    @field_validator('preferred_date')
    @classmethod
    def validate_date_format(cls, v: str) -> str:
        try:
            datetime.strptime(v, '%m/%d/%Y')
        except ValueError:
            raise ValueError('Date must be in MM/DD/YYYY format')
        return v

    @field_validator('preferred_time')
    @classmethod
    def validate_time_format(cls, v: str) -> str:
        try:
            datetime.strptime(v, '%I:%M %p')
        except ValueError:
            raise ValueError('Time must be in HH:MM AM/PM format')
        return v

class AppointmentBooking(BaseModel):
    agent: Literal["appointment"]
    intent: Literal["book_appointment"]
    details: AppointmentDetails
    missing_fields: Optional[List[str]] = Field(default_factory=list)

In [102]:
# Define the Agent
agent = Agent(
    system_prompt=(
        "You are the ADRD care system orchestrator. Analyze each message and determine the user's intent. "
        "Make sure to take the user's information into account. Without asking for any additional information. "
        "Your response should be in this format. Rules: don't just assume the information if you're missing any information, "
        "just add to the missing fields as given."
    ),
    model=model,  
    result_type=AppointmentBooking,  
    deps_type=None,
    retries=3,
)

In [105]:
result = await agent.run("hey there can you make an appointment with dr_001 on november 12 2024 about 9am for follow up")
print(result)

RunResult(_all_messages=[SystemPrompt(content="You are the ADRD care system orchestrator. Analyze each message and determine the user's intent. Make sure to take the user's information into account. Without asking for any additional information. Your response should be in this format. Rules: don't just assume the information if you're missing any information, just add to the missing fields as given.", role='system'), UserPrompt(content='hey there can you make an appointment with dr_001 on november 12 2024 about 9am for follow up', timestamp=datetime.datetime(2024, 12, 11, 6, 13, 39, 712642, tzinfo=datetime.timezone.utc), role='user'), ModelStructuredResponse(calls=[ToolCall(tool_name='final_result', args=ArgsJson(args_json='{"agent":"ADRD Care System","intent":"book_appointment","details":{"patient_name":"Unknown","doctor_id":"dr_001","preferred_date":"11/12/2024","preferred_time":"09:00 AM","request_type":"follow_up"},"missing_fields":["patient_name"]}'), tool_id='call_mnyVYd7CRUCooSx

In [106]:
result.data

AppointmentBooking(agent='appointment', intent='book_appointment', details=AppointmentDetails(patient_name='Unknown', doctor_id='dr_001', preferred_date='11/12/2024', preferred_time='09:00 AM', request_type=<RequestType.FOLLOW_UP: 'follow_up'>), missing_fields=['patient_name'])